# Description
This notebook will contain basic code to align pharmacophores (based on recently seen paper). For the starters I will use pharmacophores from align-it (also check the reader code)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import random
import torch
import sys
import json
import pyalignit

In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
CODEDIR = "../code/common"
if not CODEDIR in sys.path:
    sys.path.append(CODEDIR)
from alignit import read_pharm_set

In [144]:

# constants

def seed_everything(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    pass

# Cell with constants
DATADIR = Path("../data")
if not DATADIR.exists():
  # DATADIR.mkdir(DATADIR)
  !gdown --id 1qnvNxd6SvhwHPxD0huTpmODB270ENs7j
  !tar -xzvf inhibitors_data.tar.gz

RANDOM_SEED = 2407
seed_everything(RANDOM_SEED)

TMP_DIR = Path("../tmp")
TMP_DIR.mkdir(exist_ok=True)

train_df = pd.read_csv(DATADIR / "train_splitted_val.csv", index_col=0)
test_df = pd.read_csv(DATADIR / "test_splitted.csv", index_col=0)

# train_df['canonical'] = train_df.Smiles.apply(smiles2canonical)
# test_df['canonical'] = test_df.Smiles.apply(smiles2canonical)

# the name of the baseline BERT model which is getting fine-tuned
SMILES_COL = "Smiles"  # "canonical" 
NFOLDS = 5

PHARM_DIR = TMP_DIR / "pharmacophores" / "alignit"
print(list(PHARM_DIR.iterdir()))
phar_files = list(PHARM_DIR.iterdir())
MOL_INFO = TMP_DIR / "pharmacophores" / "molecules" / "conformers_info.json"
with open(MOL_INFO.as_posix()) as f:
    mol_info = json.load(f)
mol_info = {k: np.asarray(v) for k, v in mol_info.items()}



[PosixPath('../tmp/pharmacophores/alignit/alignit_test.phar'), PosixPath('../tmp/pharmacophores/alignit/alignit_train.phar')]


In [145]:
file_data = dict()
for file in phar_files:
    with open(file.as_posix()) as f:
        file_data[file.name] = list(read_pharm_set(f))

In [146]:
train_pharmacophores = file_data["alignit_train.phar"]
test_pharmacophores = file_data["alignit_test.phar"]

In [147]:
train_df.loc[:, "pharmacophore"] = ""
test_df.loc[:, "pharmacophore"] = ""
train2pharm = {
    index: train_pharmacophores[i]
    for i, index in enumerate(mol_info["train"])
}
test2pharm = {
    index:test_pharmacophores[i]
    for i, index in enumerate(mol_info["test"])
}


In [148]:
p = train_pharmacophores[0]
q = train_pharmacophores[1]

In [163]:
from scipy.optimize import linear_sum_assignment

def get_distance(p, q):
    coords_p = (p.x, p.y, p.z)
    coords_q = (q.x, q.y, q.z)
    d = np.sum([(pc - qc)**2 for pc, qc in zip(coords_p, coords_q)])
    return np.sqrt(d)

def describe_pharmacophore_neigborhood(p):
    # print(p.phar)
    props_neigbors = []
    for prop1 in p.phar:
        neigbor_list = []
        
        for i, prop2 in enumerate(p.phar):
            d = get_distance(prop1, prop2)
            #if d > 0:
            # print(d)
            neigbor_list.append(d)
            # type_list.append(prop1.name == prop2.name)
        #neigbor_list = sorted(neigbor_list, key=lambda x: x[0])
        props_neigbors.append(neigbor_list)
        # props_types.append(type_list)
    return np.asarray(props_neigbors)

def process_one(p, q):
    assert len(p.phar)==1
    prop0 = p.phar[0]
    q_ids = [i for i, qprop in enumerate(q.phar) if prop0.name == qprop.name]
    if len(q_ids) == 0:
        return None
    dist_list = []
    for q_id in q_ids:
        ap = prop0.a
        aq = q.phar[q_id].a
        d = np.abs(ap - aq)
        dist_list.append((q_id, d))
    dist_list = sorted(dist_list, key=lambda x: x[1])
    q_id, cost = dist_list[0]
    return [0], [q_id], cost, []



def compute_distance_matrix(features_p, features_q, p, q):
    distance_matrix = []
    ntypes = []
    for k, dp in enumerate(features_p):
        dist_list = []
        type_list = []
        for m, dq in enumerate(features_q):
            # print(dp, dq, p.phar[k], q.phar[m])
            ap = p.phar[k].a
            aq = q.phar[m].a
            d = np.abs(dp - dq)*ap*aq/(ap+aq)
            dist_list.append(d)
            type_list.append((p.phar[k].name == q.phar[m].name) and (d < 1))
        ntypes.append(type_list)
        distance_matrix.append(dist_list)
    return np.asarray(distance_matrix), np.asarray(ntypes)


def get_neighborhood_diff(p, q):
    distances_p  = describe_pharmacophore_neigborhood(p)
    distances_q  = describe_pharmacophore_neigborhood(q)
    # print(distances_p,"\n\n", distances_q)
    psize = len(p.phar)
    qsize = len(q.phar)
    size = max(psize, qsize)
    table = []
    for i, prop_p in enumerate(p.phar):
        line = []
        for j, prop_q in enumerate(q.phar):
            if prop_p.name != prop_q.name:
                line.append(np.inf)
                continue
            distances, ntypes = compute_distance_matrix(distances_p[i], distances_q[j], p, q)
            #print(distances, "\n---\n", ntypes)
            distances[~ntypes] = np.inf
            reachable_cols = np.where((distances < np.inf).any(0))[0]
            reachable_rows = np.where((distances < np.inf).any(1))[0]
            #print(distances)
            dm = distances[reachable_rows][:, reachable_cols]
            
            row_ind, col_ind = linear_sum_assignment(dm)

            min_cost = dm[row_ind, col_ind].sum()/size
            # aligned_indices = reachable_rows[row_ind], reachable_cols[col_ind]
            line.append(min_cost)
        table.append(line)
    return np.asarray(table)

    
def build_neighborhood_matrix(p, q):
    if len(p.phar) == 1:
        return process_one(p, q)
    if len(q.phar) == 1:
        return process_one(q, p)
    try:
        table = get_neighborhood_diff(p, q)
    except Exception as e:
        # print("Matrix error:", e)
        return None
    if len(table.shape) < 2:
        return None
    reachable_cols = np.where((table < np.inf).any(0))
    if len(reachable_cols) < 1:
        return None
    reachable_cols = reachable_cols[0]
    reachable_rows = np.where((table < np.inf).any(1))
    if len(reachable_rows) < 1:
        return None
    reachable_rows = reachable_rows[0]
    subtable = table[reachable_rows][:, reachable_cols]
    try:
        row_ind, col_ind = linear_sum_assignment(subtable)
    except:
        return None
    top3_indices = np.argsort(subtable[row_ind, col_ind])
    row_ind = row_ind[top3_indices]
    col_ind = col_ind[top3_indices]
    # print(row_ind, col_ind, subtable, reachable_rows, reachable_cols)
    return reachable_rows[row_ind], reachable_cols[col_ind], subtable[row_ind, col_ind], subtable
    # distances[np.eye(distances.shape[0]) == 1] = np.inf

result = build_neighborhood_matrix(p, q)
if result is None:
    p_ids, q_ids, costs, subtable = [], [], [], []
else:
    p_ids, q_ids, costs, subtable = result

p_features = [p.phar[i] for i in p_ids]
q_features = [q.phar[i] for i in q_ids]
p_matrix = np.asarray([[p.x, p.y, p.z] for p in p_features])
q_matrix = np.asarray([[q.x, q.y, q.z] for q in q_features])
print(np.sqrt(np.mean((p_matrix - q_matrix)**2)))
p_centroid = p_matrix.mean(0)
p_matrix -= p_centroid
q_centroid = q_matrix.mean(0)
q_matrix -= q_centroid


2.577107535412155


In [164]:
result = build_neighborhood_matrix(p, q)
if result is None:
    p_ids, q_ids, costs, subtable = [], [], [], []
else:
    p_ids, q_ids, costs, subtable = result
p_ids, q_ids, costs, subtable

(array([1, 4, 0]),
 array([0, 2, 1]),
 array([0.02890202, 0.03575805, 0.16170719]),
 array([[0.20473045, 0.16170719,        inf],
        [0.02890202, 0.0627753 ,        inf],
        [0.17127669, 0.18256629,        inf],
        [       inf,        inf, 0.06616799],
        [       inf,        inf, 0.03575805]]))

In [165]:
covariance_matrix = np.dot(p_matrix.T, q_matrix)

In [166]:
u, s, vh = np.linalg.svd(covariance_matrix)

In [167]:
sign = np.sign(np.linalg.det(u @ vh))
#sign = (np.linalg.det(u) * np.linalg.det(vh)) < 0.0
s[-1] = sign
s = np.array([1, 1, sign])
rot_matrix = np.dot((u*np.array([1, 1, sign])), vh)

In [168]:
q_matrix 

array([[ 3.43932233,  0.43697633,  0.03335433],
       [ 0.25750733,  0.50890133,  0.19955133],
       [-3.69682967, -0.94587767, -0.23290567]])

In [169]:
np.sqrt(np.mean((np.dot(p_matrix, rot_matrix) - q_matrix)**2))

1.2872907954210564

In [170]:
# from scipy.spatial.transform import Rotation # align_vectors
np.sqrt(np.mean((p_matrix - q_matrix)**2))

2.558211199521013

In [171]:
# train_pharmacophores[:3]
pharm_targets = train_df.Active.values[mol_info['train']]

In [172]:
pos_ids = np.where(pharm_targets)[0]
neg_ids = np.where(pharm_targets==0)[0]

In [173]:
pos_pharmacophores = [train_pharmacophores[i] for i in pos_ids]
neg_pharmacophores = [train_pharmacophores[i] for i in neg_ids]

In [174]:
result = build_neighborhood_matrix(p, q)
if result is None:
    p_ids, q_ids, costs, subtable = [], [], [], []
else:
    p_ids, q_ids, costs, subtable = result

In [175]:
train_neighborhood = dict()
for i, p in enumerate(tqdm(pos_pharmacophores)):
    for j, q in enumerate(pos_pharmacophores[i+1:]):
        # print(len(p.phar), len(q.phar))
        result = build_neighborhood_matrix(p, q)
        # if len(p.phar)==1 or len(q.phar) == 1:
        #     print(result)
        if result is not None:
            train_neighborhood[(i, j)] = result

100%|██████████| 217/217 [01:38<00:00,  2.20it/s]


In [183]:
from scipy.sparse import dok_matrix
n = len(pos_pharmacophores)
dist = dok_matrix((n, n))
for (i, j), info in train_neighborhood.items():
    if info is None:
        continue
    if isinstance(info[2], float):
        dist[i, j] = info[2]
        dist[j, i] = info[2]
        continue
    if len(info[2]) == 0:
        continue
    dist[i, j] = np.mean(info[2])
    dist[j, i] = np.mean(info[2])



In [186]:
from sklearn.cluster import AgglomerativeClustering
clusters = AgglomerativeClustering()
clusters.fit(dist)
clusters.labels_

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [162]:
test_neighborhood = dict()
for i, p in enumerate(tqdm(pos_pharmacophores[:10])):
    for j, q in enumerate(test_pharmacophores):
        # print(len(p.phar), len(q.phar))
        result = build_neighborhood_matrix(p, q)
        # if len(p.phar)==1 or len(q.phar) == 1:
        #     print(result)
        if result is not None:
            test_neighborhood[(i, j)] = result

 18%|█▊        | 39/217 [04:51<22:10,  7.47s/it]


KeyboardInterrupt: 

In [49]:
def get_features(phar):
    data = {}
    for p in phar.phar:
        if not p.name in data:
            data[p.name] = 0
        data[p.name]+=1
    return data
    pass
from tqdm.auto import tqdm
# features_df = []
# for phar in tqdm(pos_pharmacophores):
#     data = get_features(phar)
#     data["active"] = 1
#     features_df.append(data)
# for phar in tqdm(neg_pharmacophores):
#     data = get_features(phar)
#     data["active"] = 0
#     features_df.append(data)

# df = pd.DataFrame(features_df).fillna(0)
# df.head()
train_features = []
for i in train_df.index:
    if not i in train2pharm:
        train_features.append({})
        continue
    features = get_features(train2pharm[i])
    train_features.append(features)
    
pd.DataFrame(train_features)

,HYBL,HDON,HYBH,HACC,NEGC,POSC
0,2.0,1.0,1.0,NaN,NaN,NaN
1,2.0,NaN,1.0,NaN,NaN,NaN
2,3.0,1.0,1.0,3.0,NaN,NaN
3,5.0,2.0,NaN,1.0,NaN,NaN
4,4.0,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5753,4.0,NaN,1.0,3.0,1.0,1.0
5754,3.0,2.0,1.0,NaN,NaN,NaN
5755,2.0,NaN,1.0,NaN,NaN,NaN
5756,5.0,NaN,1.0,NaN,NaN,NaN


In [83]:
df.groupby("active").describe()

HYBL                                                  HDON  \
         count      mean       std  min  25%  50%  75%   max   count   
active                                                                 
0       5306.0  3.025631  1.676050  0.0  2.0  3.0  4.0  16.0  5306.0   
1        205.0  2.829268  1.560895  0.0  2.0  3.0  4.0   8.0   205.0   

                  ... POSC         NEGC                                     \
            mean  ...  75%  max   count      mean       std  min  25%  50%   
active            ...                                                        
0       1.042782  ...  0.0  6.0  5306.0  0.081040  0.332118  0.0  0.0  0.0   
1       0.858537  ...  0.0  4.0   205.0  0.180488  0.561519  0.0  0.0  0.0   

                  
        75%  max  
active            
0       0.0  6.0  
1       0.0  4.0  

[2 rows x 48 columns]

In [77]:
(df > 0).mean()

HYBL    0.946341
HDON    0.463415
HYBH    0.502439
HACC    0.663415
POSC    0.131707
NEGC    0.126829
dtype: float64